In [1]:
# lib para http
import requests

# lib para minerar os dados
from bs4 import BeautifulSoup

# lib para modelagem de dados
import pandas as pd

# lib para analise grafica
import seaborn as sns

In [2]:
# carregar a pagina

# url para buscar os dados
site = "https://www.ssp.sp.gov.br/Estatistica/ViolenciaMulher.aspx"

pagina = requests.get(site)

In [5]:
# o BS$ vai interpretar o arquivo html
coleta = BeautifulSoup(pagina.text, 'html.parser')

In [7]:
type(coleta)

bs4.BeautifulSoup

In [8]:
coleta.title

<title>
	SSP
</title>

In [12]:
# localizando as tabelas pela classe
tables = coleta.find_all('table', attrs={'class' :'table table-striped table-hover table-condensed'})

# verificando
print(f'Foram localizadas { len(tables) } tabelas')

Foram localizadas 134 tabelas


In [18]:
# exemplo para buscar
tables[0].find_all('tr')[2].find_all('td')[4].find_all('span')[0].text

'21'

In [27]:
print(i, linha)

133 <tr>
<td>HOMICÍDIO CULPOSO</td><td>0</td><td>0</td><td>0</td><td>
<span id="conteudo_repPeriodo_grdOcorrencias_135_lblTotal_1">0</span>
</td>
</tr>


In [46]:
# funcao para as datas
import datetime
from dateutil.relativedelta import relativedelta

# variaveis
rotulos = []
dados = []
periodo = []
data = []

# registro inicial do site
registro_inicial = datetime.datetime(2022, 6, 1)

# loop nas tabelas
for i, c in enumerate(tables):

  linha = c.find_all('tr')[2]

  rotulo = linha.find_all('td')[4]

  #nomenclatura feminicidio
  label = linha.find_all('td')[0].text

  informacao = rotulo.find_all('span')[0].text

  # calculando a data
  data = registro_inicial - relativedelta(months=i)

  # salvando a informacao
  dados.append(informacao)
  rotulos.append(label)
  periodo.append(data)  

  # print(i, label)

# organizando os dados
dicionario = {
    'Indicador': rotulos,
    'Valores' : dados,
    'Periodo': data
}  

# criar a tabela

tabela_web = pd.DataFrame(dicionario)

tabela_web.head()

,Indicador,Valores,Periodo
0,FEMINICÍDIO,21,2011-05-01
1,FEMINICÍDIO,13,2011-05-01
2,FEMINICÍDIO,16,2011-05-01
3,FEMINICÍDIO,20,2011-05-01
4,FEMINICÍDIO,11,2011-05-01


In [53]:
tabela_web.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Indicador  134 non-null    object        
 1   Valores    134 non-null    object        
 2   Periodo    134 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(2)
memory usage: 3.3+ KB
